Imports

In [9]:
from os import getenv

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sqlalchemy as db
from dotenv import load_dotenv

load_dotenv()

True

Connect to Database

In [3]:
# DB connection
cnx = db.create_engine(f"postgresql://{getenv('USER')}:{getenv('PASS')}@{getenv('HOST')}:{getenv('PORT')}/{getenv('DB')}")

# Player core stats

In [12]:
# Extract data based on player id
player_id = '5f3d8fdd95f40596eae23dcf'
with open('queries/player_core_stats.sql') as f:
    sql = f.read()
sql = sql.format(where=f"where gp.id = '{player_id}'")
df = pd.read_sql(sql, cnx)

## Plot data
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
for item in set(df.columns) - {'month'}:
    if item in {'avg_game_score', 'monthly_games'}:
        fig.add_trace(go.Scatter(x=df['month'], y=df[item], mode='lines', name=item), secondary_y=True)
    else:
        fig.add_trace(go.Scatter(x=df['month'], y=df[item], mode='lines', name=item))

# Add figure title
player_name = cnx.execute(f"select tag from rocket_league.players where id='{player_id}'").fetchall()[0][0]
fig.update_layout(title_text=f'Core stats for {player_name}')


fig